In [1]:
# Get Soup
import requests
from bs4 import BeautifulSoup
import re
import dateutil.parser
from urllib.request import urlopen
import pandas as pd
import numpy as np

In [2]:
## Get data from commentary url. Lowest level
def get_commentary_data(url):
    browser.open(url)
    soup = BeautifulSoup(browser.response.text,'lxml')

    ##Get author
    asses = soup.findAll('a')
    for a in asses:
        try:
            if 'author' in a['class']:
                author.append(a.text)
                break
        except:
            continue
            
    ##Get title
    title = soup.find('h1').text
    titles.append(title)

    ##Get article date
    datepublished.append(soup.find('time').text.title())

    ##Get text
    mydivs = soup.findAll('div')
    for div in mydivs:
        try:
            if div['itemprop'] == 'articleBody':
                article_div = div
        except:
            continue

    text = ""
    for i in article_div.findAll('p'):
        text += i.text

    articles.append(text)

def commentarylistscrape(topicurl):
    browser.open(topicurl)
    soup = BeautifulSoup(browser.response.text,'lxml')
    commentaries = soup.findAll('h2')
    for commentary in commentaries:
        try:
            if "commentary" in commentary.find('a')['href']:
                commentarylist.append('https://www.project-syndicate.org'+commentary.find('a')['href'])
                topicmarker.append(topicurl.replace('http://www.project-syndicate.org/',''))
        except:
            continue

In [62]:
### Get urls of all archive pages
response = requests.get('https://www.project-syndicate.org/archive')
soup = BeautifulSoup(response.text,'lxml')

max_page = int(soup.find(text=re.compile('pages')).strip().split()[0])

archive_pages_url = []
for x in range(1,max_page+1):
    archive_pages_url.append('https://www.project-syndicate.org/archive?page='+str(x))

response = requests.get(archive_pages_url[0])
soup = BeautifulSoup(response.text,'lxml')

commentarylist=[]
topicmarker = []

for archive in archive_pages_url:
    commentarylistscrape(archive)

In [20]:
author=[]
datepublished = []
articles = []
titles = []

## Robobrowser
from robobrowser import RoboBrowser

browser = RoboBrowser()
browser.open('http://project-syndicate.org/login')

# Get the signup form
signup_form = browser.get_form(class_='login user-input-form')
# signup_form         # <RoboForm user[name]=, user[email]=, ...

# # Inspect its values
# signup_form['authenticity_token'].value     # 6d03597 ...

# # Fill it out
signup_form['email'].value = 'kenndanielso@gmail.com'
signup_form['password'].value = '123qweQWE'

# # Submit the form
browser.submit_form(signup_form)

## Extract Data
for commentary in commentarylist:
    get_commentary_data(commentary)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [25]:
author = pd.Series(author)
date = pd.Series(datepublished)
article = pd.Series(articles)
title = pd.Series(titles)
source = pd.Series(commentarylist)

ps_df = pd.concat((title,author,date,article,source),axis=1)

ps_df.columns = ['title','author','date','article','source']

In [26]:
ps_df

,title,author,date,article,source
0,The Gulf States’ Expat Dividend,Sami Mahroum,"Jun 17, 2016",PARIS – How should policymakers in the Middle ...,https://www.project-syndicate.org/commentary/g...
1,Health-Care Innovation in the Global South,Shams El Arifeen,"Jun 17, 2016",DHAKA – Children die from preventable and trea...,https://www.project-syndicate.org/commentary/d...
2,A Good Economy for China,Edmund S. Phelps,"Jun 17, 2016",NEW YORK – Decades of plodding growth together...,https://www.project-syndicate.org/commentary/c...
3,Brexit’s Impact on the World Economy,Anatole Kaletsky,"Jun 17, 2016",LONDON – The febrile behavior of financial mar...,https://www.project-syndicate.org/commentary/b...
4,Europe’s Refugee Culture Clash,Paulina Neuding,"Jun 16, 2016",STOCKHOLM – The international Christian organi...,https://www.project-syndicate.org/commentary/e...
5,Gays on the Front Line,Bernard-Henri Lévy,"Jun 16, 2016","PARIS – The mass shooting in Orlando, Florida,...",https://www.project-syndicate.org/commentary/o...
6,The World’s Ins and Outs,Lucy P. Marcus,"Jun 16, 2016",LONDON – As the United Kingdom’s debate about ...,https://www.project-syndicate.org/commentary/b...
7,Toward a More Reflective Planet,David Keith,"Jun 16, 2016",CAMBRIDGE – The last time the atmosphere held ...,https://www.project-syndicate.org/commentary/a...
8,The Agony of the Republicans,Elizabeth Drew,"Jun 16, 2016","WASHINGTON, DC – This is a grim time for Ameri...",https://www.project-syndicate.org/commentary/t...
9,What’s the Matter With Austria?,Dalia Marin,"Jun 15, 2016","MUNICH – Last month, Austria avoided the elect...",https://www.project-syndicate.org/commentary/a...


In [27]:
ps_df.to_pickle('ps_df_2.pkl')